In [2]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device("cuda")

In [3]:
import sys
sys.path.append('./huggingface_models/')
sys.path.append('./utils/')
from sample_utils import *
from inference_utils import *
from codenet_process_utils import *
from self_training_utils import *

In [4]:
%load_ext autoreload
%autoreload 2

### Filtering:
1. remove duplicated preds: get_dedup_preds
2. filter by type-matching: prep_exec_hypo_codenet
3. filter by compilation: get_hypo_call_list

We get hypo call_dict in this step. \
call_list contains info about the processed hypos in lang2.\
```
call_list = [programs, processed_results, result_keys, error_type_dict]
call_dict[(lang1, lang2)] = [new_preds, functions, function_id_dict, call_list]
```

### Cached Files
- Hypo call_dict\
    plbart_codenet_lang_pair_call_dict.pkl


### Load No-tok Preprocessed Data

In [5]:
is_plbart = True
merged_filtered_dict = get_prepro_filtered_dict(None, is_plbart)
programs_dict = get_codenet_programs_dict(merged_filtered_dict)

#### Get Import Str

In [6]:
import_str_dict = {}
for lang in new_langs:
    all_imports, import_str = get_common_imports(lang, merged_filtered_dict)
    import_str_dict[lang] = import_str
import_str_dict["Java"] = java_imports_str
import_str_dict["C#"] = csharp_imports_str

### Merge Preds

In [11]:
preds_lang_dict_all = {}
for lang in new_langs:
    for i in [1, 2, ""]:
        dic_path = cached_path + "codet5_full_" + lang + "_codenet_" + str(i) + "preds_lang_dict.pkl"
        if not os.path.exists(dic_path):
            continue
        print(dic_path)
        with open(dic_path, 'rb') as infile:
            preds_lang_dict_batch = pickle.load(infile)
            for k, v in preds_lang_dict_batch.items():
                print(lang, k)
                preds_lang_dict_all[k] = v
print(preds_lang_dict_all.keys())

/home/mingzhu/CodeModel/CodeGen_cwd/cached_files/codet5_full_C++_codenet_2preds_lang_dict.pkl
C++ ('C++', 'C#')
/home/mingzhu/CodeModel/CodeGen_cwd/cached_files/codet5_full_Java_codenet_1preds_lang_dict.pkl
Java ('C#', 'Java')
Java ('C#', 'Python')
Java ('C#', 'C++')
Java ('Java', 'C#')
Java ('Java', 'Python')
Java ('Java', 'C++')
/home/mingzhu/CodeModel/CodeGen_cwd/cached_files/codet5_full_Java_codenet_preds_lang_dict.pkl
Java ('C', 'Java')
Java ('C', 'Python')
Java ('C', 'C#')
Java ('C', 'C++')
Java ('Java', 'C')
/home/mingzhu/CodeModel/CodeGen_cwd/cached_files/codet5_full_Python_codenet_preds_lang_dict.pkl
Python ('C', 'Java')
Python ('C', 'Python')
Python ('C', 'C#')
Python ('C', 'C++')
Python ('Java', 'C')
Python ('Python', 'C')
/home/mingzhu/CodeModel/CodeGen_cwd/cached_files/codet5_full_C#_codenet_1preds_lang_dict.pkl
C# ('C#', 'Java')
C# ('C#', 'Python')
C# ('C#', 'C++')
/home/mingzhu/CodeModel/CodeGen_cwd/cached_files/codet5_full_C#_codenet_preds_lang_dict.pkl
C# ('C', 'Java')

In [10]:
print(len(preds_lang_dict_all.keys()))

14


In [12]:
plbart_sample_path = cached_path + 'codet5_full_codenet_preds_lang_dict.pkl'
with open(plbart_sample_path, 'wb') as outfile:
     pickle.dump(preds_lang_dict_all, outfile)

### Load Pre-Sampled Data

In [5]:
plbart_sample_path = cached_path + 'plbart_codenet_preds_lang_dict.pkl'
with open(plbart_sample_path, 'rb') as infile:
     preds_lang_dict_plbart = pickle.load(infile)

In [7]:
codet5_sample_path = cached_path + 'codet5_codenet_preds_lang_dict.pkl'
with open(codet5_sample_path, 'rb') as infile:
     preds_lang_dict_codet5 = pickle.load(infile)

In [ ]:
codet5_sample_path = cached_path + 'codet5_codenet_preds_lang_dict.pkl'
with open(codet5_sample_path, 'rb') as infile:
     preds_lang_dict_codet5 = pickle.load(infile)

### Filter Generated Hypotheses by Type-Matching and Compilation

#### Get call_dict for Hypotheses

In [ ]:
lang_pairs = [('Python', 'Java'), ('Python', 'C#'),
            ('Java', 'C#'), ('Java', 'C++'), ('Java', 'C'), 
            ('C#', 'Java'), ('C#', 'Python'), ('C#', 'C++'), ('C#', 'C')]
call_dict = {}
for lang1, lang2 in lang_pairs:
    print(lang1, lang2)
    preds = preds_lang_dict_all[(lang1, lang2)]
    # remove duplicated preds
    new_preds = get_dedup_preds(preds)
    # filter by type-matching (Todo)
    functions, function_id_dict = prep_exec_hypo_codenet(new_preds, lang1, lang2, 
                                                                 merged_filtered_dict, model_type)
    # filter by compilation
    call_list = get_hypo_call_list(functions, lang2, import_str_dict)
    call_dict[(lang1, lang2)] = [new_preds, functions, function_id_dict, call_list]
    with open(cached_path + "codenet_lang_pair_call_dict.pkl", 'wb') as outfile:
        pickle.dump(call_dict, outfile)

In [12]:
with open(cached_path + "plbart_full_codenet_lang_pair_call_dict.pkl", 'rb') as infile:
    call_dict = pickle.load(infile)

#### Get all the filtered hypos 

In [13]:
filtered_lang_dict = {}
for lang1, lang2 in call_dict.keys():
    new_preds, functions, function_id_dict, call_list = call_dict[(lang1, lang2)] 
    filtered_dict = get_compiled_hypos(call_list, function_id_dict, merged_filtered_dict)
    filtered_lang_dict[(lang1, lang2)] = filtered_dict

NameError: name 'merged_filtered_dict' is not defined

#### Separate pids that have filtered hypos and pids that don't

In [ ]:
empty_dict = {}
non_empty_dict = {}
for lang1, lang2 in filtered_lang_dict.keys():
    non_empty_dict[(lang1, lang2)] = []
    empty_dict[(lang1, lang2)] = []
    filtered_dict = filtered_lang_dict[(lang1, lang2)]
    for pid, inds in filtered_dict.items():
        if len(inds) > 0:
            non_empty_dict[(lang1, lang2)].append(pid)
        else:
            empty_dict[(lang1, lang2)].append(pid)
    print(lang1, lang2, len(non_empty_dict[(lang1, lang2)]))

### Resampling for pids that doesn't have filtered hypo

In [ ]:
def sampling_filtering_compilation(pids, eval_examples, eval_features, function_id_lang_dic,
                       reverse_map_dict, result_id_dict_gold, lang, model_type, model, tokenizer, args, device,
                       decoder_sid=None, is_eval=True, num_samples=5, temperature=0.5):
    selected_eval_examples, selected_eval_dataloader = get_eval_data_by_pid(eval_examples, eval_features, 
                                                               pids, reverse_map_dict, args.eval_batch_size)
    torch.cuda.empty_cache()
    preds, eval_result = generation_multiple(selected_eval_examples, 
                                             selected_eval_dataloader, 
                                             model, tokenizer, args, device, 
                                             decoder_sid, is_eval, num_samples, temperature)
    programs, program_id_dict, program_dict = prep_exec_hypo(preds, pids,
                                                             function_id_lang_dic, lang, model_type)
    lang_results = p_map(file_executors[lang], programs)
    result_id_dict, result_key_dict, error_type_dict = result_mapping(lang_results, program_id_dict, 
                                                                      pids, lang)
    buggy_pids, failed_test_pids, passed_hypo_dict = hypo_filtering(result_id_dict, 
                                                                    result_id_dict_gold, 
                                                                    result_key_dict, 
                                                                    program_dict)
    print(len(buggy_pids),len(failed_test_pids), len(passed_hypo_dict), len(pids))
    return buggy_pids, failed_test_pids, passed_hypo_dict

In [ ]:
# resamples for pids in empty_dict

In [30]:
lang1 = "C#"
lang2 = "C++"
preds = preds_lang_dict_all[(lang1, lang2)][:100]
new_preds = get_dedup_preds(preds)
functions, function_id_dict = prep_exec_hypo_codenet(new_preds, lang1, lang2, 
                                                            merged_filtered_dict, model_type)
# call_list = get_hypo_call_list(functions, lang2)
# filtered_dict = get_compiled_hypos(call_list, function_id_dict, merged_filtered_dict)

100%|██████████| 100/100 [00:01<00:00, 76.87it/s]


In [32]:
call_list = get_hypo_call_list(functions, lang2)


C++ error 348 0.8169014084507042
C++ timeout 0 0.0
C++ empty 78 0.18309859154929578
C++ other 0 0.0
C++ good 0 0.0


In [ ]:
filtered_dict = get_compiled_hypos(call_list, function_id_dict, merged_filtered_dict)

In [ ]:
for i, (error_key, error)  in enumerate(zip(result_keys, processed_results)):
    if error_key == "error":
        if "Compilation failed" in error:
            print(i, error)
            break